In [1]:
#Import Data Manipulation Libraries
import pandas as pd
import numpy as np
#import Data Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt
#import Filter Warnings Library
import warnings
warnings.filterwarnings("ignore")
#import Logging Library
import logging
logging.basicConfig(level=logging.INFO,
                    filename="model.log",
                    filemode="w",
                    format="%(asctime)s - %(levelname)s - %(message)s",force=True)

In [ ]:
#load the dataset 

url=""